#### Задача 1 (5 баллов).

Реализуйте с помощью классов базу данных для ветеринарной клиники: нам нужно хранить сведения о питомцах, их хозяевах и датах приема. Видимо, нам понадобятся два класса: для владельцев животных и для самих животных. У владельца мы храним имя, фамилию и номер телефона. У животного пусть будет кличка, возраст и дополнительный атрибут "записан на прием", в котором будет храниться информация о дате следующего приема (если ее нет - None). База данных должна храниться в бинарном файле (используйте pickle или dill). При запуске программы должны быть доступны несколько действий: завести нового клиента, завести новое животное для существующего клиента и записать животное на прием (или отметить, что прием прошел и новой записи нет). Сами экземпляры клиентов хранятся просто в списке.

Примечание: в атрибутах могут храниться не только строки, но и списки, но не забывайте об их особенностях.

In [ ]:
import pickle
from dataclasses import dataclass

class Database:
    def __init__(self):
        self.owners: list[Owner] = []

class Owner:
    def __init__(self, first_name: str, last_name: str,
                 phone_number: str):
        self.first_name: str = first_name
        self.last_name: str = last_name
        self.phone_number: str = phone_number
        self.pets: list[Pet] = []

    def __str__(self):
        return f"Владелец: {self.first_name} {self.last_name}\n\
Телефон: {self.phone_number}\n\
Питомцы: {self.pets}"


class Pet:
    def __init__(self, name: str, age: int,
                 at_appointment: str = None):
        self.name: str = name
        self.age: int = age
        self.at_appoitment: str | None = at_appointment

    def __str__(self):
        return f"Питомец: {self.name},\n\t\
Возраст: {self.age}\n\t\
Прием: {self.at_appoitment}\n\t"

    def __repr__(self):
        return f"Питомец: {self.name},\n\t\
Возраст: {self.age}\n\t\
Прием: {self.at_appoitment}\n\t"


def choose_owner(db) -> int:
    print("Выберите клиента цифрой:")
    for i, owner in enumerate(db.owners):
        print(f"{i + 1}: {owner.first_name} {owner.last_name}")
    while True:
        try:
            choise = int(input())
            if choise > len(db.owners):
                print("Нет такого клиента, попробуйте еще раз")
                continue
            return choise - 1
        except ValueError:
            print("Вы ввели не цифру, попробуйте еще раз")
            continue

def choose_pet(owner: Owner) -> int:
    print("Выберите питомца цифрой:")
    for i, pet in enumerate(owner.pets):
        print(f"{i + 1}: {pet.name}")
    while True:
        try:
            choise = int(input())
            if choise > len(owner.pets):
                print("Нет такого клиента, попробуйте еще раз")
                continue
            break
        except ValueError:
            print("Вы ввели не цифру, попробуйте еще раз")
    return choise - 1

In [1]:
import os
import pickle


running_state = True
path = os.path.join("HW04", "database.pkl")

if os.path.exists(path):
    db = pickle.load(open(path, "rb"))
else:
    db = Database()

while running_state:
    user_input = input("Введите команду:\n")
    match user_input.lower():
        case "помощь":
            print("Команды:\n\t\
Помощь\n\t\
Ввести клиента\n\t\
Добавить питомца\n\t\
Удалить питомца\n\t\
Вывести\n\t\
Выйти")
        case "выйти":
            pickle.dump(db, open(path, "wb"))
            running_state = False
        case "ввести клиента":
            user_input = input("Введите имя клиента, фамилию,\
и номер телефона через запятую:\n")
            owner_data = user_input.split(", ")
            owner = Owner(*owner_data)
            db.owners.append(owner)
        case "добавить питомца":
            choise = choose_owner(db)
            print("Введите имя питомца, возраст и дату осмотра (если есть) через запятую")
            user_input = input()
            pet_data = user_input.split(", ")
            pet = Pet(*pet_data)
            db.owners[choise].pets.append(pet)
        case "удалить питомца":
            choise = choose_owner(db)
            pet_choise = choose_pet(db.owners[choise])
            db.owners[choise].pets.pop(pet_choise)
        case "изменить дату осмотра":
            choise = choose_owner(db)
            pet_choise = choose_pet(db.owners[choise])
            user_input = input("Введите дату осмотра или нажмите энтер для удаления")
            if user_input == "\n":
                user_input = None
            db.owners[choise].pets[pet_choise].at_appoitment = user_input
        case "вывести":
            for owner in db.owners:
                print()
                print(owner)
        case "удалить клиента":
            choise = choose_owner(db)
            db.owners.pop(choise)

Введите команду:

Владелец: Иван Иванович
Телефон: 89998887766
Питомцы: [Питомец: Муся,
	Возраст: 10
	Прием: 25.12.2024
	]

Владелец: Артем Артемович
Телефон: 89991112233
Питомцы: [Питомец: Гарик,
	Возраст: 3
	Прием: None
	]
Введите команду:
Введите имя клиента, фамилию,и номер телефона через запятую:
Введите команду:

Владелец: Иван Иванович
Телефон: 89998887766
Питомцы: [Питомец: Муся,
	Возраст: 10
	Прием: 25.12.2024
	]

Владелец: Артем Артемович
Телефон: 89991112233
Питомцы: [Питомец: Гарик,
	Возраст: 3
	Прием: None
	]

Владелец: а а
Телефон: 1
Питомцы: []
Введите команду:
Выберите клиента цифрой:
1: Иван Иванович
2: Артем Артемович
3: а а
Введите команду:

Владелец: Иван Иванович
Телефон: 89998887766
Питомцы: [Питомец: Муся,
	Возраст: 10
	Прием: 25.12.2024
	]

Владелец: Артем Артемович
Телефон: 89991112233
Питомцы: [Питомец: Гарик,
	Возраст: 3
	Прием: None
	]
Введите команду:


#### Задача 2 (5 баллов).

Хотим написать программу - базу данных для библиотеки. Нам понадобится класс "книга", который будет содержать автора, название, жанр и количество страниц; а также класс "библиотека", в котором в атрибутах будут сидеть все наши книги. В библиотеку будут поступать запросы: нужно будет выдавать перечень всех книг одного автора, перечень всех книг конкретного жанра, а также проверять, что книга такого автора и с таким названием есть (вам хорошо помогут генераторные выражения для всех этих вещей, а еще можно переопределить магический метод \_\_eq\_\_ у класса "книга" - он должен возвращать bool, а внутри него можно сравнивать атрибуты экземпляров self и other - и проверять наличие объекта класса "книга" в списке). Магический метод \_\_eq\_\_ неявным образом вызывается, когда вы сравниваете два объекта: например, сравниваете две книжки. Он будет неявно вызываться и тогда, когда вы проверяете наличие какого-то объекта в списке таких же объектов с помощью оператора in. 

In [ ]:
# your code here
class Book:
    def __init__(self, name: str, author: str, genre: str, pages: int):
        self.name: str = name
        self.author: str = author
        self.genre: str = genre
        self.pages: int = pages

    def __eq__(self, other):
        return self.name.lower() == other.name.lower() and self.author.lower() == other.author.lower()

    def __repr__(self):
        return f"Book({self.name}, {self.author}, {self.genre}, {self.pages})"

    def __str__(self):
        return f"Название: {self.name}\n\t\
Автор: {self.author}\n\t\
Жанр: {self.genre}\n\t\
Страниц: {self.pages}"


class Library:
    def __init__(self):
        self.books = []

    def add_book(self, book: Book):
        self.books.append(book)

    def find_by_author(self, author):
        return [book for book in self.books if book.author.lower() == author.lower()]

    def find_by_genre(self, genre):
        return [book for book in self.books if book.genre.lower() == genre.lower()]

    def has_book(self, name, author):
        book = Book(name, author, "", 0)
        return book in self.books

In [ ]:
lib = Library()
lib.add_book(Book("Оно", "Стивен Кинг", "триллер", 1138))
lib.add_book(Book("Молчание ягнят", "Томас Харрис", "триллер", 352))
lib.add_book(Book("Generation П", "Виктор Олегович Пелевин", "сатира", 384))
lib.add_book(Book("Чапаев и Пустота", "Виктор Олегович Пелевин", "сатира", 370))
lib.add_book(Book("Дверь в лето", "Роберт Хайнлайн", "фантастика", 288))
lib.add_book(Book("Звездный дисант", "Роберт Хайнлайн", "фантастика", 352))
lib.add_book(Book("451 градус по Фаренгейту", "Рэй Бредберри", "антиутопия", 304))
lib.add_book(Book("Унесенные ветром Т. 1", "Маргарет Митчелл", "драма", 624))
lib.add_book(Book("Унесенные ветром Т. 2", "Маргарет Митчелл", "драма", 592))

In [2]:
while True:
    print("Выберите опцию:")
    print("1: Найти по автору")
    print("2: Найти по жанру")
    print("3: Проверить наличие")
    print("4: Выйти")
    try:
        user_input = int(input())
    except ValueError:
        print("Вы ввели не цифру, попробуйте снова")
        continue
    match user_input:
        case 1:
            author = input("Введите автора")
            for book in lib.find_by_author(author):
                print(book)
        case 2:
            genre = input("Введите жанр")
            for book in lib.find_by_genre(genre):
                print(book)
        case 3:
            name = input("Введите имя")
            author = input("Введите жанр")
            print(lib.has_book(name, author))
        case 4:
            break
        case _:
            print("Нет такой опции")

Выберите опцию:
1: Найти по автору
2: Найти по жанру
3: Проверить наличие
4: Выйти
Введите автора[]
Выберите опцию:
1: Найти по автору
2: Найти по жанру
3: Проверить наличие
4: Выйти
Введите автора[]
Выберите опцию:
1: Найти по автору
2: Найти по жанру
3: Проверить наличие
4: Выйти
Вы ввели не цифру, попробуйте снова
Выберите опцию:
1: Найти по автору
2: Найти по жанру
3: Проверить наличие
4: Выйти


#### Задача 3 (5 баллов). 

Представим себе, что мы хотим написать игрушку, в которой главный персонаж ходит по карте и с каждым броском кубика ему выпадают разнообразные предметы, с которыми он может взаимодействовать. Нам, очевидно, понадобится хранить все результаты взаимодействия в классе персонажа, а также написать хотя бы три различных предмета, которые умеют делать разные вещи (пусть будет волшебное зелье, книжка, прибавляющая мудрость, и медвежий капкан; можете что-то свое придумать). У персонажа есть свои характеристики, которые тоже могут влиять на результат взаимодействия с вещами (очень тупой персонаж может не смочь прочитать книжку...). 

Бонусный балл: напишите класс для карты, у которого будет метод с собственно циклом, где кидается наш "кубик" и персонаж перемещается, подбирая предметы. 

Для рандомизации используем библиотеку random. Также можно использовать функцию sleep из библиотеки time: эта функция заставляет компьютер ждать x секунд. 

In [ ]:
from random import randint # функция генерирует любое случайное число от х до у: randint(x, y)
from time import sleep # sleep(1) - ждать 1 секунду
from os import path


class Hero:
    def __init__(self):
        self.health: int = 100
        self.iq: int = 10
        # если кончатся ноги, то персонаж умер, ноги теряются в капканах
        self.legs: int = 2
        # шанс уклона от капкана в процентах
        self.dodge: int = 0
        # как додж, но для зелий
        self.knowledge: int = 0
        self.killed: bool = False

    def __repr__(self):
        return f"Hero({self.health}, {self.iq})"

    def interact(self, obj):
        return obj.use(self)

    def __str__(self):
        return f"ОЗ: {self.health}\n\
Интеллект: {self.iq}\n\
Ноги: {self.legs}\n\
Уклонение: {self.dodge}\n\
Распознaвание зелий: {self.knowledge}"


class Potion:
    def __init__(self, strength: int, coef: float):
        self.strength: int = round(strength * coef)

    # Не уверен, как передать коэф в __repr__
    # Как видно, я вначале хотел сделать его статическим, но в итоге он у меня в экземпляре класса Map
    # Так же не хочется создавать доп переменную в __init__ по типу self.coef
    def __repr__(self):
        return f"Potion({self.strength}, {Map.coef})"

    def use(self, hero: Hero):
        print(f"Вы нашли зелье")
        sleep(1)
        if hero.knowledge >= randint(0, 100):
            return f"Вы распознали зелье отравления на {self.strength} урона и не стали его пить!"
        hero.health -= self.strength
        if hero.health <= 0:
            print("Вы умерли от отравления!")
            hero.killed = True
        return f"Вы отравились на {self.strength} ОЗ, у Вас осталось {hero.health} ОЗ!"


class Book:
    def __init__(self, iq: int):
        self.iq: int = iq

    def __repr__(self):
        return f"Book({self.iq})"

    def use(self, hero: Hero):
        print(f"Вы читаете книгу сложностью {self.iq}")
        sleep(2)
        if hero.iq >= self.iq:
            book_type = randint(0, 2)
            if book_type == 0:
                hero.iq += 5
                return "Вы прочитали книгу и повысили интеллект на 5!"
            if book_type == 1:
                hero.dodge = min(50, hero.dodge + 5)
                return f"Вы прочитали книгу о ловушках, Ваш шанс уклонения от капкана: {hero.dodge}%"
            hero.knowledge = min(50, hero.knowledge + 5)
            return f"Вы прочитали книгу о зельях, Ваш шанс распознать зелье: {hero.knowledge}%"
        return "Вы ничего не поняли, книга была слишком сложная"


class Trap:
    def __init__(self, strength: int, coef: float):
        self.strength: int = round(strength * coef * 2)

    def use(self, hero: Hero):
        print("Вы нашли капкан")
        sleep(1)
        if hero.dodge >= randint(0, 100):
            return "Вы успешно уклонились от капкана!"
        hero.legs -= 1
        if hero.legs == 0:
            hero.killed = True
            return "Вы умерли без ног!"
        hero.health -= self.strength
        if hero.health <= 0:
            hero.killed = True
        return f"Вы угодили в капкан, получили {self.strength} урона и потеряли ногу!"

    def __repr__(self):
        return f"Trap({self.strength}, {Map.coef})"


class FirstAidKit:
    def __init__(self, regen: int, coef: float):
        self.regen: int = round(regen * coef * 2)

    def use(self, hero: Hero):
        print("Вы нашли аптечку!")
        sleep(1)
        if hero.legs == 1:
            hero.legs += 1
            return "Вы восстановили ногу!"
        hero.health += self.regen
        return f"Вы восстановили {self.regen} ОЗ!"

    def __repr__(self):
        return f"FirstAidKit({self.regen}, {Map.coef})"


class Map:
    def __init__(self):
        if path.exists(path.join("HW04", "record.txt")):
            with open(path.join("HW04", "record.txt"), "r") as f:
                self.record = int(f.read())
        else:
            self.record = 0
        self.hero: Hero = Hero()
        self.coef: int = 0.1
        self.moves = 0

    def travel(self):
        print("Начинаем игру!")
        print(f"Рекорд: {self.record}")
        while True:
            print()
            if self.hero.killed:
                print("Игра окончена!")
                print(f"Ваш счет: {self.moves} ходов")
                if self.moves > self.record:
                    with open(path.join("HW04", "record.txt"), "w") as f:
                        f.write(str(self.moves))
                break
            print(f"{'Ход: ' + str(self.moves + 1):^10}")
            print("Ваши статы:")
            print(self.hero)

            #когда поймете игру, советую поставить везде sleep(1)
            sleep(2)
            rn = randint(0, 3)
            match rn:
                case 0:
                    obj = Potion(randint(10, 50), self.coef)
                case 1:
                    obj = Trap(randint(20, 50), self.coef)
                case 2:
                    obj = Book(randint(0, 40))
                case 3:
                    obj = FirstAidKit(randint(10, 75), self.coef)

            print()
            print(self.hero.interact(obj))
            self.coef += 0.05
            self.moves += 1
            sleep(2)

In [3]:
map = Map()
map.travel()

Начинаем игру!
Рекорд: 32

  Ход: 1  
Ваши статы:
ОЗ: 100
Интеллект: 10
Ноги: 2
Уклонение: 0
Распознaвание зелий: 0

Вы нашли капкан
Вы угодили в капкан, получили 10 урона и потеряли ногу!

  Ход: 2  
Ваши статы:
ОЗ: 90
Интеллект: 10
Ноги: 1
Уклонение: 0
Распознaвание зелий: 0

Вы нашли капкан
Вы умерли без ног!

Игра окончена!
Ваш счет: 2 ходов
